In [226]:
import scipy.io
import numpy as np
import pandas as pd

# read correlation matrices
df=np.zeros((200,90,8))

for i in range(8):
   filename='C:/Users/liang/JGMSS_python/data/%s_ts_mean.mat' % (i+1)
   data=scipy.io.loadmat(filename)
   df[:,:,i]=pd.DataFrame(data['mean_ts'])

global n
global K
global p

[n,p,K]=df.shape


In [227]:
def center(time_series):
    [n,p]=time_series.shape
    mu=np.mean(time_series)
    time_series=time_series-np.ones((n,1))*mu
    return time_series, mu

In [228]:
def normalize(time_series):
    time_series=df[:,:,1]
    [n,p]=time_series.shape
    [time_series, mu]=center(time_series)
    d=np.sqrt(np.sum(np.multiply(time_series,time_series),axis=0))
    d[np.where(d == 0)]=1
    time_series.shape
    np.ones((n,1)).shape
    d=np.reshape(d,(1,90))
    d.shape
    #dd
    #np.dot(np.ones(n,1),d)
    time_series=np.divide(time_series,np.dot(np.ones((n,1)),d))
    return time_series,mu,d


In [229]:
def objective(S,X,Z,lambda1,lambda2):
    [p,p,K]=Z.shape
    sum1=np.zeros((p,p))
    sum1=np.sum(Z,2)
    sum1_sqrt=np.sqrt(sum1)
    
    M1=np.identity(p)
    M2=np.ones((p,p))
    M3=np.subtract(M2,M1)
    sum1_sqrt=np.multiply(sum1_sqrt,M3)
    N=n/2
    
    #L1 part
    sumL1=0;
    Z1=np.zeros((p,p))
    for m in range(K):
        Z1=Z[:,:,m]
        M1=np.identity(p)
        M2=np.ones(p)
        M3=np.subtract(M2,M1)
        Z1=np.multiply(Z1,M3)
        sumL1=sumL1+np.linalg.norm(Z1,ord=1)
        
    sumL1=sumL1/K
    
    #L12 part
    import numpy.ma as ma
    X1=np.zeros((p,p,K))
    E=np.zeros((K,1))
    for i in range(K):
        mask=np.abs(X)>1e-05
        X1=ma.array(X,mask=mask)
        E[i]=np.count_nonzero(X1[:,:,i])
    sum2=0;
    for i in range(K):
        sum2=sum2+np.trace(np.dot(S[:,:,i],X[:,:,i]))-np.log(np.linalg.det(X[:,:,i]))
    t1=sum2
    obj=N*t1+lambda1*sumL1+lambda2*np.linalg.norm(sum1_sqrt,1)
    t2=2*np.sum(E)
    
    AIC=N*t1+t2
    BIC=N*t1+np.log(N)*t2/2
    return obj, AIC, BIC

In [230]:
def shrinkage(a,kappa):
    y=np.maximum(0,a-kappa)-np.maximum(0,-a-kappa)
    return y

In [252]:
def JGM(D,lambda1,lambda2, rho, alpha):
    from numpy import linalg as LA
    MAX_ITER=4000
    ABSTOL=1e-4
    RELTOL=1e-2
    
    [n,p,K]=D.shape
    S=np.zeros((p,p,K))
    for i in range(K):
        S[:,:,i]=np.cov(D[:,:,i].transpose())
    
    #ADMM implementation
    X=np.zeros((p,p,K))
    Z=np.zeros((p,p,K))
    U=np.zeros((p,p,K))
    
    sum_shrinkage=np.zeros((p,p))
    es=np.zeros((p,K))
    Q=np.zeros((p,p,K))
    L=np.zeros((p,p,K))   
    xi=np.zeros((p,K))
    Zold=np.zeros((p,p,K))
    X_hat=np.zeros((p,p,K))
    temp=np.zeros((p,p,K))
    ind_record=0
    
    class history:
        r_norm=np.zeros((MAX_ITER,K))
        s_norm=np.zeros((MAX_ITER,K))
        eps_pri=np.zeros((MAX_ITER,K))
        eps_dual=np.zeros((MAX_ITER,K))
        objval=np.zeros((MAX_ITER))
        AIC=np.zeros(MAX_ITER)
        BIC=np.zeros(MAX_ITER)
    
    for i in range(MAX_ITER):
        sum_shrinkage=np.zeros((p,p))
        for k in range(K):
            #x-update
            tt=(-Z[:,:,k]+U[:,:,k])/n+S[:,:,k]
            Q[:,:,k],L[:,:,k]=LA.eig(rho*(Z[:,:,k]-U[:,:,k])/n-S[:,:,k])
            es[:,k]=np.diag(L[:,:,k])
            xi[:,k]=n*(es[:,k]+np.sqrt(np.multiply(es[:,k],es[:,k])+4*rho/n))/(2*rho)
            X[:,:,k]=np.matmul(np.matmul(Q[:,:,k],np.diag(xi[:,k])),Q[:,:,k].transpose())
            
            #z-update
            Zold[:,:,k]=Z[:,:,k]
            X_hat[:,:,k]=alpha*X[:,:,k]+(1-alpha)*Zold[:,:,k]
            temp[:,:,k]=shrinkage(X_hat[:,:,k]+U[:,:,k],lambda1/rho)
            sum_shrinkage[:,:]=sum_shrinkage[:,:]+np.multiply(temp[:,:,k],temp[:,:,k])
        sum_shrinkage=sum_shrinkage+0.01*np.ones((p,p))
        
        for k in range(K):
            Z[:,:,k]=np.multiply(shrinkage(X_hat[:,:,k] + U[:,:,k],lambda1/rho),np.maximum(1-lambda2/(rho*np.sqrt(sum_shrinkage[:,:])),0))
            
            U[:,:,k]=U[:,:,k]+(X_hat[:,:,k]-Z[:,:,k])
            
            history.r_norm[i,k]=LA.norm(X[:,:,k]-Z[:,:,k],'fro')
            history.s_norm[i,k]=LA.norm(-rho*(Z[:,:,k]-Zold[:,:,k]),'fro')
            
            history.eps_pri[i,k] = np.sqrt(n*n)*ABSTOL + RELTOL*np.maximum(LA.norm(X[:,:,k],'fro'), LA.norm(Z[:,:,k],'fro'));
            history.eps_dual[i,k]= np.sqrt(n*n)*ABSTOL + RELTOL*LA.norm(rho*U[:,:,k],'fro')
        
            if history.r_norm[i,k] < history.eps_pri[i,k] and history.s_norm[i,k] < history.eps_dual[i,k]:
                ind_record=i
                break
            
        if ind_record==i:
            history.objval[i],history.AIC[i], history.BIC[i]  = objective(S,X,Z,lambda1,lambda2)
            break
        else:
            history.objval[i],history.AIC[i],history.BIC[i]  = objective(S,X,Z,lambda1,lambda2) 
            
    return Z, history

In [253]:
#Normalize time series
TS_normalized_all=np.zeros((200,90,8))
for i in range(8):
   [TS_normalized_all[:,:,i],mu,d]=normalize(df[:,:,i])


In [254]:
#Divide each dataset into 20 blocks, 5 consecutive time points for each block


for i in range(20):
    exec('TS_block%s' % i + '=' +'TS_normalized_all[%s:%s,:,:]' % (i,i+5))
    

TS_block1=np.zeros((5,90,8))
TS_block2=np.zeros((5,90,8))
TS_block3=np.zeros((5,90,8))
TS_block4=np.zeros((5,90,8))
TS_block5=np.zeros((5,90,8))
TS_block6=np.zeros((5,90,8))
TS_block7=np.zeros((5,90,8))
TS_block8=np.zeros((5,90,8))
TS_block9=np.zeros((5,90,8))
TS_block10=np.zeros((5,90,8))
TS_block11=np.zeros((5,90,8))
TS_block12=np.zeros((5,90,8))
TS_block13=np.zeros((5,90,8))
TS_block14=np.zeros((5,90,8))
TS_block15=np.zeros((5,90,8))
TS_block16=np.zeros((5,90,8))
TS_block17=np.zeros((5,90,8))
TS_block18=np.zeros((5,90,8))
TS_block19=np.zeros((5,90,8))
TS_block20=np.zeros((5,90,8))


TS_block1[0:4,:,:]=TS_normalized_all[0:4,:,:]
TS_block2[1:5,:,:]=TS_normalized_all[6:10,:,:]
TS_block3[1:5,:,:]=TS_normalized_all[11:15,:,:]
TS_block4[1:5,:,:]=TS_normalized_all[16:20,:,:]
TS_block5[1:5,:,:]=TS_normalized_all[21:25,:,:]
TS_block6[1:5,:,:]=TS_normalized_all[26:30,:,:]
TS_block7[1:5,:,:]=TS_normalized_all[31:35,:,:]
TS_block8[1:5,:,:]=TS_normalized_all[36:40,:,:]
TS_block9[1:5,:,:]=TS_normalized_all[41:45,:,:]
TS_block10[1:5,:,:]=TS_normalized_all[46:50,:,:]
TS_block11[1:5,:,:]=TS_normalized_all[51:55,:,:]
TS_block12[1:5,:,:]=TS_normalized_all[56:60,:,:]
TS_block13[1:5,:,:]=TS_normalized_all[61:65,:,:]
TS_block14[1:5,:,:]=TS_normalized_all[66:70,:,:]
TS_block15[1:5,:,:]=TS_normalized_all[71:75,:,:]
TS_block16[1:5,:,:]=TS_normalized_all[76:80,:,:]
TS_block17[1:5,:,:]=TS_normalized_all[81:85,:,:]
TS_block18[1:5,:,:]=TS_normalized_all[86:90,:,:]
TS_block19[1:5,:,:]=TS_normalized_all[91:95,:,:]
TS_block20[1:5,:,:]=TS_normalized_all[96:100,:,:]

In [256]:
#Tuning hyperparameters
import random
vec=np.zeros((100,10))
Y=np.zeros((p,p,100,5,10))
I=np.zeros((p,p,K,100,5,10))
for lambda1 in range(5):
    for lambda2 in range(10):
        #Radom subsampling
        for itr in range(100):
            vec[itr,:]=random.sample(range(20),10)
            var1='TS_block%s' % vec[itr,0].astype(int)
            var2='TS_block%s' % vec[itr,1].astype(int)
            var3='TS_block%s' % vec[itr,2].astype(int)
            var4='TS_block%s' % vec[itr,3].astype(int)
            var5='TS_block%s' % vec[itr,4].astype(int)
            var6='TS_block%s' % vec[itr,5].astype(int)
            var7='TS_block%s' % vec[itr,6].astype(int)
            var8='TS_block%s' % vec[itr,7].astype(int)
            var9='TS_block%s' % vec[itr,8].astype(int)
            var10='TS_block%s' % vec[itr,9].astype(int)
            
            #Subsampled data at group-level
            data=np.concatenate([eval(var1),eval(var2),eval(var3),eval(var4),eval(var5),eval(var6),eval(var7),eval(var8),eval(var9),eval(var10)])
            
            #JGMSS
            X, history =JGM(data, lambda1*0.01, 0.08+lambda2*0.07, 1.0, 1.0)
            
            temp=np.zeros((p,p))
            
            
            for i in range(p):
                for j in range(p):
                    if np.absolute(X[i,j,0])>0 and np.absolute(X[i,j,1])>0 and np.absolute(X[i,j,2])>0 and  np.absolute(X[i,j,3])>0 and np.absolute(X[i,j,4])>0 and np.absolute(X[i,j,5])>0 and np.absolute(X[i,j,6])>0 and np.absolute(X[i,j,7])>0:
                        temp[i,j]=1;
                    else:
                        temp[i,j]=0
                        
            #output variables Y & I
            Y[:,:,itr, lambda1, lambda2]=temp[:,:]
            I[:,:,:,itr,lambda1,lambda2]=X[:,:,:]
                    
            #number ofo nonzeros elements, variable S: stable matrix
            S=np.zeros((p,p,5,10))
            for i in range(p):
                for j in range(p):
                    S[i,j,lambda1,lambda2]=np.sum(Y[i,j,:,lambda1, lambda2])/100
            
            #save variables S, Y & I
            filepath_S='C:/Users/liang/JGMSS_Python/S.npy'
            np.save(filepath_S,S)
            filepath_Y='C:/Users/liang/JGMSS_Python/Y.npy'
            np.save(filepath_Y,Y)
            filepath_I='C:/Users/liang/JGMSS_Python/I.npy'
            np.save(filepath_I,I)

C:\Users\liang\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\liang\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log
